In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

dataset = pd.read_csv("./../../TushareStock/Stocks/t_000001", sep='\t',  names=["date", "open", "high","close","low","volume","price_change","p_change","ma5","ma10","ma20","v_ma5","v_ma10","v_ma20","turnover"], header=0).sort_values('date',ascending=True)
print(dataset.head(5))

data = dataset[[ 'close', 'volume', 'open', 'high', 'low']].values.astype(float)
print(data[0:5, :])
data = np.flipud(data)
print(data.shape)

lookback_period = 30
data_matrix = np.empty([(data.shape[0] - lookback_period), data.shape[1], lookback_period])
print(data_matrix.shape)

scaler = MinMaxScaler(feature_range=(-1, 1))

for i in range(data_matrix.shape[0]): # for each example
    for j in range(data_matrix.shape[1]): # for each feature
        scaler.fit(data[i: i + lookback_period, j].reshape(lookback_period, 1))

data_matrix[i, j, :] = scaler.transform(data[i: i + lookback_period, j].reshape(1, -1))
data_matrix = np.swapaxes(data_matrix, 1, 2)
print(data_matrix.shape)

 # create y values: 1 if close at day 30 > close at day 29. Else 0.
def up_down(yest, tod):
    if tod >= yest:
        return 1
    else:
        return 0
perm = np.random.permutation(data_matrix.shape[0])
data_matrix = data_matrix[perm]
targets = np.empty([data_matrix.shape[0], 1])
for i in range(data_matrix.shape[0]):
    targets[i] = up_down(data_matrix[i][-2][0], data_matrix[i][-1][0])

           date   open   high  close    low      volume  price_change  \
607  2018-05-02  10.97  11.03  10.88  10.80  1190523.25          0.03   
606  2018-05-03  10.86  10.88  10.75  10.57  1281355.62         -0.13   
605  2018-05-04  10.73  10.83  10.68  10.66   710509.50         -0.07   
604  2018-05-07  10.70  10.83  10.81  10.64   974309.69          0.13   
603  2018-05-08  10.83  11.15  11.01  10.80  1373305.62          0.20   

     p_change     ma5    ma10    ma20       v_ma5      v_ma10      v_ma20  \
607      0.28  10.880  10.880  10.880  1190523.25  1190523.25  1190523.25   
606     -1.20  10.815  10.815  10.815  1235939.44  1235939.44  1235939.44   
605     -0.65  10.770  10.770  10.770  1060796.12  1060796.12  1060796.12   
604      1.22  10.780  10.780  10.780  1039174.52  1039174.52  1039174.52   
603      1.85  10.826  10.826  10.826  1106000.74  1106000.74  1106000.74   

     turnover  
607      0.70  
606      0.76  
605      0.42  
604      0.58  
603      0.81  
[[

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_matrix[:, :-1, :], targets, stratify=targets, test_size=0.2)
'''
layers: 1 LSTM (32 units)
1 Dense (1 unit)
lookback_period = 30
'''
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2]), stateful=False, return_sequences=True))
model.add(LSTM(8, input_shape=(x_train.shape[1], x_train.shape[2]), stateful=False))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics = ['accuracy'])
EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20)
model.fit(x_train, y_train, batch_size=20, validation_split = 0.20, epochs=100, shuffle=False)

 # baseline accuracy (= accuray if you always chose the most frequent y-value in testset)
baseline_acc= float(max(sum(y_test)/len(y_test), (1 - sum(y_test)/len(y_test))))
print("Baseline accuracy: " + str(baseline_acc))

# LSTM accuracy
loss_and_metrics = model.evaluate(x_test, y_test)
lstm_acc = float(loss_and_metrics[1])
print("LSTM accuracy: " + str(lstm_acc))
# LSTM premium
premium = lstm_acc - baseline_acc
print("LSTM premium: " + str( premium))
# a = pd.DataFrame.from_dict(baseline_acc, orient='index').rename(columns = {0: "baseline_acc"})
# b = pd.DataFrame.from_dict(lstm_acc, orient='index').rename(columns = {0: "lstm_acc"})
# c = pd.DataFrame.from_dict(premium, orient='index').rename(columns = {0: "premium"})
# result = pd.concat([a, b, c], axis=1)
# result

Train on 369 samples, validate on 93 samples
Epoch 1/100
369/369 [==============================] - 2s 6ms/step - loss: 0.6328 - acc: 0.9268 - val_loss: 0.5306 - val_acc: 0.9570
Epoch 2/100
369/369 [==============================] - 1s 1ms/step - loss: 0.3061 - acc: 0.9810 - val_loss: 0.1604 - val_acc: 0.9570
Epoch 3/100
369/369 [==============================] - 1s 1ms/step - loss: 0.1130 - acc: 0.9810 - val_loss: 0.1166 - val_acc: 0.9570
Epoch 4/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0888 - acc: 0.9810 - val_loss: 0.1048 - val_acc: 0.9462
Epoch 5/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0824 - acc: 0.9864 - val_loss: 0.1016 - val_acc: 0.9570
Epoch 6/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0809 - acc: 0.9864 - val_loss: 0.1032 - val_acc: 0.9570
Epoch 7/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0812 - acc: 0.9837 - val_loss: 0.1148 - val_acc: 0.9462
Epoch 8/100
369/369 [===

Epoch 62/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0812 - acc: 0.9810 - val_loss: 0.1242 - val_acc: 0.9570
Epoch 63/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0811 - acc: 0.9810 - val_loss: 0.1243 - val_acc: 0.9570
Epoch 64/100
369/369 [==============================] - 0s 1ms/step - loss: 0.0812 - acc: 0.9810 - val_loss: 0.1244 - val_acc: 0.9570
Epoch 65/100
369/369 [==============================] - 0s 1ms/step - loss: 0.0811 - acc: 0.9810 - val_loss: 0.1245 - val_acc: 0.9570
Epoch 66/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0812 - acc: 0.9810 - val_loss: 0.1247 - val_acc: 0.9570
Epoch 67/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0811 - acc: 0.9810 - val_loss: 0.1244 - val_acc: 0.9570
Epoch 68/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0810 - acc: 0.9810 - val_loss: 0.1234 - val_acc: 0.9570
Epoch 69/100
369/369 [==============================] - 1s 1ms